<a href="https://colab.research.google.com/github/judygab/Machine-Learning/blob/master/Symptoms_descriptions_tfhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a text classification model with TF Hub

In this notebook, we'll walk you through building a model to predict the symptoms and conditions of a patient given his description. The emphasis here is not on accuracy, but instead how to use TF Hub layers in a text classification model.

To start, import the necessary dependencies for this project.

In [33]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import json
import pickle
import urllib

from sklearn.preprocessing import MultiLabelBinarizer

print(tf.__version__)

1.14.0


## The dataset

We need a lot of text inputs to train our model. For this model we'll use [this medical symptom classification](https://www.kaggle.com/paultimothymooney/medical-symptoms-text-and-audio-classification) from Kaggle. To simplify things I've made the `overview_of_recordings.csv` file available on my GitHub it can be downloaded from the url. 

In [0]:
# Download the data from GitHub
url = 'https://raw.githubusercontent.com/judygab/Data-for-ML-models/master/overview-of-recordings.csv'

Next we'll convert the dataset to a Pandas dataframe and print first 5 rows. For this model we're only using 2 of these columns: `phrase` and `prompt`.

In [35]:
data = pd.read_csv(url)
data.head()

,audio_clipping,audio_clipping:confidence,background_noise_audible,background_noise_audible:confidence,overall_quality_of_the_audio,quiet_speaker,quiet_speaker:confidence,speaker_id,file_download,file_name,phrase,prompt,writer_id
0,no_clipping,1.0000,light_noise,1.0000,3.33,audible_speaker,1.0,43453425,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43453425_58166571.wav,When I remember her I feel down,Emotional pain,21665495
1,light_clipping,0.6803,no_noise,0.6803,3.33,audible_speaker,1.0,43719934,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43719934_43347848.wav,When I carry heavy things I feel like breaking...,Hair falling out,44088126
2,no_clipping,1.0000,no_noise,0.6655,3.33,audible_speaker,1.0,43719934,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43719934_53187202.wav,there is too much pain when i move my arm,Heart hurts,44292353
3,no_clipping,1.0000,light_noise,1.0000,3.33,audible_speaker,1.0,31349958,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_31349958_55816195.wav,My son had his lip pierced and it is swollen a...,Infected wound,43755034
4,no_clipping,1.0000,no_noise,1.0000,4.67,audible_speaker,1.0,43719934,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43719934_82524191.wav,My muscles in my lower back are aching,Infected wound,21665495


## Preparing the data for our model



In [0]:
phrases = data['phrase']
prompts = data['prompt']
# urllib.request.urlretrieve('https://storage.googleapis.com/bq-imports/descriptions.p', 'descriptions.p')
# urllib.request.urlretrieve('https://storage.googleapis.com/bq-imports/genres.p', 'genres.p')

# descriptions = pickle.load(open('descriptions.p', 'rb'))
# symptoms = pickle.load(open('symptoms.p', 'rb'))

### Splitting our data
When we train our model, we'll use 80% of the data for training and set aside 20% of the data to evaluate how our model performed.

In [0]:
train_size = int(len(phrases) * .8)

train_phrases = phrases[:train_size].astype('str')
train_prompts = prompts[:train_size]

test_phrases = phrases[train_size:].astype('str')
test_prompts = prompts[train_size:]

### Formatting our labels
When we train our model we'll provide the labels (in this case prompts) associated with each phrase. We can't pass the genres in as strings directly, we'll transform them into multi-hot vectors.

In [38]:
encoder = MultiLabelBinarizer()
encoder.fit_transform(train_prompts)
train_encoded = encoder.transform(train_prompts)
test_encoded = encoder.transform(test_prompts)
num_classes = len(encoder.classes_)

# Print all possible genres and the labels for the first phrase in our training dataset
print(encoder.classes_)
print(train_encoded[0])

[' ' 'A' 'B' 'C' 'E' 'F' 'H' 'I' 'J' 'K' 'M' 'N' 'O' 'S' 'a' 'b' 'c' 'd'
 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'r' 's' 't' 'u' 'v' 'w'
 'y' 'z']
[1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 0 0 1 0 0 0 0
 0]


### Create our TF Hub embedding layer
[TF Hub]() provides a library of existing pre-trained model checkpoints for various kinds of models (images, text, and more) In this model we'll use the TF Hub `universal-sentence-encoder` module for our pre-trained word embeddings. When we train our model, it'll convert our array of patient description strings to embeddings. When we train our model, we'll use this as a feature column.


In [0]:
phrase_embeddings = hub.text_embedding_column("phrases", module_spec="https://tfhub.dev/google/universal-sentence-encoder/2", trainable=False)


## Instantiating our DNNEstimator Model
The first parameter we pass to our DNNEstimator is called a head, and defines the type of labels our model should expect. Since we want our model to output multiple labels, we’ll use multi_label_head here. Then we'll convert our features and labels to numpy arrays and instantiate our Estimator. `batch_size` and `num_epochs` are hyperparameters - you should experiment with different values to see what works best on your dataset.

In [0]:
multi_label_head = tf.contrib.estimator.multi_label_head(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

In [41]:
features = {
  "phrases": np.array(train_phrases).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)
train_input_fn = tf.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=32, num_epochs=25)
estimator = tf.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,10],
    feature_columns=[phrase_embeddings])

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpv5qs8uzk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8a3ea20128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpv5qs8uzk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8a3ea20128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training and serving our model 
To train our model, we simply call `train()` passing it the input function we defined above. Once our model is trained, we'll define an evaluation input function similar to the one above and call `evaluate()`. When this completes we'll get a few metrics we can use to evaluate our model's accuracy.


In [42]:
estimator.train(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:loss = 0.38435304, step = 4101 (0.838 sec)


INFO:tensorflow:Saving checkpoints for 4163 into /tmp/tmpv5qs8uzk/model.ckpt.


INFO:tensorflow:Saving checkpoints for 4163 into /tmp/tmpv5qs8uzk/model.ckpt.


INFO:tensorflow:Loss for final step: 0.3865968.


INFO:tensorflow:Loss for final step: 0.3865968.


In [43]:
# Define our eval input_fn and run eval
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"phrases": np.array(test_phrases).astype(np.str)}, test_encoded.astype(np.int32), shuffle=False)
estimator.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving dict for global step 4163: auc = 0.8683915, auc_precision_recall = 0.7182744, average_loss = 0.37097135, global_step = 4163, loss = 0.37146065


INFO:tensorflow:Saving dict for global step 4163: auc = 0.8683915, auc_precision_recall = 0.7182744, average_loss = 0.37097135, global_step = 4163, loss = 0.37146065


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4163: /tmp/tmpv5qs8uzk/model.ckpt-4163


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4163: /tmp/tmpv5qs8uzk/model.ckpt-4163


{'auc': 0.8683915,
 'auc_precision_recall': 0.7182744,
 'average_loss': 0.37097135,
 'global_step': 4163,
 'loss': 0.37146065}

## Generating predictions on new data
Let's generate predictions on patient descriptions our model hasn't seen before. We'll define an array of 3 new description strings (the comments indicate the correct genres) and create a `predict_input_fn`. Then we'll display the top 2 prompts along with their confidence percentages for each of the 3 phrases.

In [0]:
# Test our model on some raw description data
raw_test = [
    "I have a head ache", # Head Ache
]



In [0]:
# Generate predictions
predict_input_fn = tf.estimator.inputs.numpy_input_fn({"phrases": np.array(raw_test).astype(np.str)}, shuffle=False)
results = estimator.predict(predict_input_fn)

In [46]:
# Display predictions
for symptom_descriptions in results:
  top_2 = symptom_descriptions['probabilities'].argsort()[-2:][::-1]
  for symptom in top_2:
    text_symptom = encoder.classes_[symptom]
    print(text_symptom + ': ' + str(round(symptom_descriptions['probabilities'][symptom] * 100, 2)) + '%')
  print('')

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


 : 92.26%
e: 79.15%

